In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenmake
from snakenmake import *

In [ ]:
def plot_cell(cell):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 12))
    for layer_datatype, polys in poly_dict.items():
        # if layer_datatype[0] == 2:
        #    continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="r", fc="none")
    plt.axes().set_aspect("equal", "datalim")

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([23e3, 13e3])

In [ ]:
fov = 6.5 * 2048 / 40
fov

In [ ]:
23e3 * 13e3 / fov**2

In [ ]:
Cell = partial(gdspy.Cell, exclude_from_current=True)

In [ ]:
lib = gdspy.GdsLibrary()

In [ ]:
from snakenmake import *

# plot_cell(profilometry_marks())
# plot_cell(alignment_cross())
plot_cell(wafer([chip("foo", snake)] * 6, "This is a \n label"))

In [ ]:
from snakenmake import *

In [ ]:
chip_cell = chip(
    "foo",
    snake,
    dims=chip_dims,
    split=5,
    margin=1.5e3,
    trench_spacing=2,
    feeding_channel_width=90,
    lane_gap=20,
)
plot_cell(chip_cell)

In [ ]:
cells, metadata = snake(
    chip_dims,
    split=5,
    margin=1.5e3,
    trench_spacing=2,
    feeding_channel_width=90,
    lane_gap=20,
)
snake_cell = cells[0]
chip_cell = Cell("Chip")
chip_cell.add(outline(chip_dims))
chip_cell.add(g.CellReference(snake_cell, (0, 0)))
plot_cell(chip_cell)
print(metadata)

In [ ]:
cells, metadata = snake(
    chip_dims, split=3, margin=1e3, trench_spacing=1.5, feeding_channel_width=90
)
snake_cell = cells[0]
chip_cell = Cell("Chip")
chip_cell.add(outline(chip_dims))
chip_cell.add(g.CellReference(snake_cell, (0, 0)))
plot_cell(chip_cell)
print(metadata)

In [ ]:
main_cell = Cell("main")
main_cell.add(gdspy.CellReference(chip_cell, (0, 0)))

In [ ]:
plot_cell(main_cell)

In [ ]:
lib = gdspy.GdsLibrary(
    infile="/Users/jacob/Dropbox/Research/Paulsson/snakenbake/test/20170426_Vibrio_Suction_W1.GDS"
)

In [ ]:
[c.name for c in lib.top_level()]

In [ ]:
main_cell = next(c for c in lib.top_level() if c.name == "main")
poly_dict = main_cell.get_polygons(by_spec=True)

In [ ]:
plt.figure(figsize=(20, 20))
for layer_datatype, polys in poly_dict.items():
    if layer_datatype[0] == 2:
        continue
    for poly in polys:
        plt.fill(*poly.T, lw=0.5, ec="r", fc="none")